#Netcat

This notebook provide some basic netcat functionality which you need to let a stream run so that the Spark Streaming Engine can process it.

The simplest way is to click `Run All` on the top. (You can change the message it sends, if you want). <br>
To do this, after you run it, you have to `stop the Execution` and you have to manually execute `nc.close()`. <br>
Then you can modify the function `Netcat.send()` and then press `Run All` again.

In [10]:
import socket
import time
from pyspark import SparkContext

class Netcat:

    """ Python 'netcat like' module """

    def __init__(self, port):
        # sc = SparkContext()
        self.ready = False
        self.port = port
        
        self.spark_master_node_ip = self.getMasterNodeIP()
        if self.spark_master_node_ip is None:
          return
        
        self.buff = ""
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.socket.bind((self.spark_master_node_ip, port))
        self.socket.listen(10)
        self.conn, addr = self.socket.accept()
       
        self.ready = True
        
    def getMasterNodeIP(self):
      from urllib.parse import urlparse
      try:
        url_parsed = urlparse(sc.uiWebUrl)
        ip = url_parsed.netloc.split(':')[0]
        return ip
      except Exception as exp:
        print(exp)
        return None
    
    def send(self):
        if self.ready is not True:
          print("Socket is not ready. Exiting.")
          return
        # RUN ONLY 3 iterations per each window of 1 second to make sure that wordcount works as expected
        i = 0
        while True:
          try:
            print("Sending iteration "+ str(i))
            self.write('Hello World Bye World')
            self.write('Hello Spark Goodbye Spark')
            self.write("ass isn't allowed.")
            i+=1
            if i == 3:
              time.sleep(1)
              i = 0
          except IOError as e:
            if e.errno == 32:
              # Broken Pipe Exception
              print("Socket was closed. Re-initializing.")
              self.close()
              self.__init__(self.port)  # re-initialize
            else:
              print(str(e))
              self.close()
              break
          except Exception as e:
            print(str(e))
            self.close()
            break
 
    def write(self, data):
        # append newline between messages
        data = data + "\n"
        # encode before sending through socket
        message = data.encode()
        self.conn.send(message)
    
    def close(self):
      self.socket.close()
      
    def __del__(self):
      self.close()

In [11]:
nc = Netcat(9993)
# start a new Netcat() instance
# Note: When you experiment with the producer you might receive the following error message: OSError: [Errno 98] Address already in use
#       A simple fix is to change the port number of the producer and the consumer, e.g. from 9999 to 9998
while True:
  try:
    nc.send()
  except Exception as e:
    print(str(e))
  finally:
    nc.close()


name 'sc' is not defined
Socket is not ready. Exiting.


AttributeError: 'Netcat' object has no attribute 'socket'

In [ ]:
# Always close at the end
nc.close()